In [1]:
from transformers import TFAutoModelForTokenClassification, ElectraTokenizerFast
from datasets import load_dataset, load_metric
import numpy as np
from transformers import DataCollatorWithPadding
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall
import tensorflow_addons as tfa
import math
from transformers import TokenClassificationPipeline
from sklearn.metrics import f1_score

2022-03-18 11:39:22.791825: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-18 11:39:22.791864: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model = TFAutoModelForTokenClassification.from_pretrained(
    "pretrained_models/NegBioElectra", num_labels=1,from_pt=True,local_files_only=True
)
tokenizer = ElectraTokenizerFast.from_pretrained("pretrained_models/NegBioElectra",max_length=256,local_files_only=True)
tokenizer.add_tokens(['[NEG]','[SPE]'], special_tokens=True)

2022-03-18 11:39:25.091064: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-18 11:39:25.091098: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-18 11:39:25.091129: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default): /proc/driver/nvidia/version does not exist
2022-03-18 11:39:25.091385: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraForTokenClassif

2

In [3]:
model.load_weights("pretrained_models/bioscope_models/NegBioElectra_bioscope_scope_model")

In [4]:
ds = load_dataset("csv", data_files={'neg_test': 'data/bioscope_leaderboard_neg_test.csv','spec_test':'data/bioscope_leaderboard_spe_test.csv'})
ds

Using custom data configuration default-043f5dd429419ac1
Reusing dataset csv (/home/studio-lab-user/.cache/huggingface/datasets/csv/default-043f5dd429419ac1/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    neg_test: Dataset({
        features: ['Unnamed: 0', 'sentence', 'labels'],
        num_rows: 216
    })
    spec_test: Dataset({
        features: ['Unnamed: 0', 'sentence', 'labels'],
        num_rows: 315
    })
})

In [5]:
def convert_labels(examples):
    l = {'labels':[list(np.array(i.split("|"),dtype="int32")) for i in examples['labels']]} 
    l.update(tokenizer(examples['sentence'],truncation=True,max_length=256,padding=True))
    return l

In [6]:
encoded_dataset = ds.map(convert_labels, batched=True)

Loading cached processed dataset at /home/studio-lab-user/.cache/huggingface/datasets/csv/default-043f5dd429419ac1/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-09ae7c8eccb8015a.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Ignored unknown kwarg option direction


In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,max_length=256, return_tensors="tf",padding="max_length")

In [8]:

tf_neg_test_dataset = encoded_dataset["neg_test"].to_tf_dataset(
    columns=['token_type_ids', 'attention_mask', 'input_ids'],
    label_cols=["labels"],
    shuffle=False,
    batch_size=1,
    collate_fn=data_collator,
)
tf_spec_test_dataset = encoded_dataset["spec_test"].to_tf_dataset(
    columns=['token_type_ids', 'attention_mask', 'input_ids'],
    label_cols=["labels"],
    shuffle=False,
    batch_size=1,
    collate_fn=data_collator,
)

In [9]:
def sigmoid_array(x):                                        
    return 1 / (1 + np.exp(-x))

In [10]:
inp = []
pred_out = []
true_out = []
for i in tf_neg_test_dataset:
    in_,out_ = list(i)
    inp += list(in_['input_ids'].numpy())
    true_out += list(out_.numpy())
    pred = model(in_)
    pred_out += list(sigmoid_array(pred['logits'].numpy()))
    
len(inp),len(true_out),len(pred_out)

(216, 216, 216)

In [11]:
pred_out = np.array(pred_out)
pred_out.shape

(216, 256, 1)

In [12]:
pred_out = np.reshape(pred_out,(pred_out.shape[0],256))
pred_out.shape

(216, 256)

In [13]:
pred_out = np.round(pred_out)
pred_out[0]

array([0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [14]:
true_out = np.array(true_out)
true_out.shape

(216, 256)

In [15]:
f1_score(true_out, pred_out, average='macro',zero_division=1)

0.989387636502077

In [16]:
inp = []
pred_out = []
true_out = []
for i in tf_spec_test_dataset:
    in_,out_ = list(i)
    inp += list(in_['input_ids'].numpy())
    true_out += list(out_.numpy())
    pred = model(in_)
    pred_out += list(sigmoid_array(pred['logits'].numpy()))
    
len(inp),len(true_out),len(pred_out)

(315, 315, 315)

In [17]:
pred_out = np.array(pred_out)
pred_out.shape

(315, 256, 1)

In [18]:
pred_out = np.reshape(pred_out,(pred_out.shape[0],256))
pred_out.shape

(315, 256)

In [19]:
pred_out = np.round(pred_out)
pred_out[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [20]:
true_out = np.array(true_out)
true_out.shape

(315, 256)

In [23]:
f1_score(true_out, pred_out, average='macro',zero_division=1)

0.9837165734789863